In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')



In [ ]:
### Read in the data

zl_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/z400_11-15_12.npy')
zl_box = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_box/z400_11-15_12.npy')
zl_hole = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_hole/z400_11-15_12.npy')

pvl_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv400_11-15_12.npy')
pvl_box = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_box/pv400_11-15_12.npy')
pvl_hole = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_hole/pv400_11-15_12.npy')

pvlay_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv300-500_11-15_12.npy')
pvlay_box = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_box/pv300-500_11-15_12.npy')
pvlay_hole = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_hole/pv300-500_11-15_12.npy')

p_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/slp_11-16_12.npy')
p_box = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_box/slp_11-16_12.npy')
p_hole = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_cape_hole/slp_11-16_12.npy')

dp_box = p_box - p_ctl
dp_hole = p_hole - p_ctl

lats = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lons.npy')


In [ ]:
### Plot subdomain
min_lat = 29
max_lat = 45
min_lon = -95
max_lon = -79

### Cartopy projection
proj = crs.PlateCarree()

### Contour intervals
zlvls = np.arange(0,1500,10)
dzlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75
plvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*1.75

fig, axd = plt.subplot_mosaic([['topleft','topright','cbar1'],
                               ['bottomleft','bottomright','cbar2']],
                              constrained_layout=True, figsize=(6.8,5.8), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,0.05],'height_ratios':[1,1]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['topleft'].contour(lons, lats, smooth2d(zl_box,10)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['topleft'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['topleft'].contourf(lons, lats, pvlay_box - pvlay_ctl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['topleft'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('CAPE Target')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)
axd['topleft'].text(-0.26, 0.78, 'Upper ($t$ = 48 h)',transform=axd['topleft'].transAxes + trans,
            fontsize=12, verticalalignment='top',rotation='vertical')

##############
### TOP RIGHT
#############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['topright'].contour(lons, lats, smooth2d(zl_hole,10)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['topright'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['topright'].contourf(lons, lats, pvlay_hole - pvlay_ctl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['topright'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('CAPE Target$^\complement$')
axd['topright'].text(0.0, 1.0, '(b)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### Colorbar
cbar1 = fig.colorbar(im1, orientation='vertical', cax=axd['cbar1'])

##############
### BOTTOM LEFT
#############

### Plot subdomain
min_lat = 31
max_lat = 49
min_lon = -79
max_lon = -61

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(p_ctl,30),
                  levels=np.arange(0,1500,6),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='k')
cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(p_box,30),
                  levels=np.arange(0,1500,6),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='magenta')
im1 = axd['bottomleft'].contourf(lons, lats, dp_box, levels=plvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomleft'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].text(0.0, 1.0, '(c)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)
axd['bottomleft'].text(-0.26, 0.82, 'Surface ($t$ = 72 h)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12, verticalalignment='top',rotation='vertical')

##############
### BOTTOM RIGHT
##############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(p_ctl,30),
                  levels=np.arange(0,1500,6),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='k')
cs1 = axd['bottomright'].contour(lons, lats, smooth2d(p_hole,30),
                  levels=np.arange(0,1500,6),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='magenta')
im1 = axd['bottomright'].contourf(lons, lats, dp_hole, levels=plvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomright'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].text(0.0, 1.0, '(d)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### Colorbar
cbar2 = fig.colorbar(im1, orientation='vertical', cax=axd['cbar2'])

plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig14_wrf_4km_cape.pdf',bbox_inches='tight')
plt.show()
